In [ ]:
import xarray as xr
from emit_tools import emit_xarray
import os
from fsspec.implementations.http import HTTPFileSystem
import holoviews as hv
import hvplot.xarray  # noqa: F401

from utils import band_index, gamma_adjust, get_rgb_dataset, build_interactive_map

## Loading data

In [ ]:
# Generate a user token and set it as an environment variable
# Tokens can be generated at https://urs.earthdata.nasa.gov/users/<username>/user_tokens
token = os.environ.get('EARTHDATA_TOKEN')
if token is None:
    raise ValueError('Please set the EARTHDATA_TOKEN environment variable')

In [ ]:
%%time
# Loading data can take around 4 minutes on a 100Mbps connection

# kernel = "EMIT_L2A_RFL_001_20230123T004529_2302216_003" # Hobart
kernel = "EMIT_L2A_RFL_001_20230316T045211_2307503_006" # Canberra

s3_url = "s3://lp-prod-protected/EMITL2ARFL.001/" + kernel + "/" + kernel + ".nc"
s3_url = s3_url.replace("s3://", "https://data.lpdaac.earthdatacloud.nasa.gov/")

fs = HTTPFileSystem(headers={
    "Authorization": f"bearer {token}"
})
ds = emit_xarray(fs.open(s3_url))
ds

## Render data as RGB and query point for spectra plot

In [ ]:
%%capture --no-stdout

# Select wavelengths to be displayed
r = 2100
g = 950
b = 490

# select brightness - range between 0-1, higher values 'brighten' the whole scene
brightness = 0.4

## End configuration area ##

# Get data structured for rendering
ds_rgb = get_rgb_dataset(ds, [r, g, b], brightness)

# Create a HoloViews RGB map and spectral plot
layout = build_interactive_map(ds, ds_rgb)

In [ ]:
# Show the interactive plot
layout

## Run normalised band index

The logic here is to pick a band where the reflectance of water is
high and where it's low, isolate those bands and do a NDWI calculation.

From [this reference](https://bmcecol.biomedcentral.com/articles/10.1186/s12898-019-0233-0/tables/2),
it's suggested to use something like `860` for the first value and
`1240`, `1640` or `2130` for the high value (short/long wavelengths). By inspecting bands, I've picked `650` and `1275`.

In [ ]:
# High meaning bright, or high reflectance
high = 450
low = 1275

ds["water_high"] = ds.reflectance.isel(bands=band_index(ds, high))
ds["water_low"] = ds.reflectance.isel(bands=band_index(ds, low))
ds["water_diff"] = ds.water_high - ds.water_low

ds["ndwi"] = ds.water_diff / (ds.water_high + ds.water_low)

In [ ]:
plot_options = {
    "aspect": "equal",
    "frame_width": 300,
    "xaxis": "bare",
    "yaxis": "bare",
}

image_high = ds.water_high.hvplot(**plot_options, title=f"high: {high}", clim=(0, 0.4), cmap="viridis")
image_low = ds.water_low.hvplot(**plot_options, title=f"low: {low}", clim=(0, 0.4), cmap="viridis")
image_diff = ds.water_diff.hvplot(**plot_options, title="difference", clim=(-1, 1), cmap="RdBu")
image_ndwi = ds.ndwi.hvplot(**plot_options, title="NDWI", clim=(-1, 1), cmap="bwy_r")


(image_high + image_low + image_diff + image_ndwi).cols(2)

In [ ]:
# Plot a histogram of the NDWI values, so we can pick a threshold
ds.ndwi.hvplot.hist()

In [ ]:
ds["threshold_water"] = ds.ndwi > 0.0
ds.threshold_water.hvplot()

In [ ]:
# Convert threshold_water band to uint8 and save to disk
ds.threshold_water.astype("uint8").rio.to_raster(f"water.tif")

## Exporting data

In [ ]:
# Export a single band as a GeoTIFF
ds.isel(bands=band_index(ds, r)).reflectance.rio.to_raster(f"band_{r}.tif")
ds.isel(bands=band_index(ds, g)).reflectance.rio.to_raster(f"band_{g}.tif")
ds.isel(bands=band_index(ds, b)).reflectance.rio.to_raster(f"band_{b}.tif")

In [ ]:
# Combine three images into a single GeoTIFF with three bands
import osgeo_utils.gdal_merge

output_file_path = f"bands_{r}_{g}_{b}.tif"
input_files_path = f"band_{r}.tif band_{g}.tif band_{b}.tif".split()
parameters = ['', '-o', output_file_path] + input_files_path + ['-separate', '-co', 'COMPRESS=LZW']
osgeo_utils.gdal_merge.main(parameters)
